In [1]:
import pandas as pd
from settings import *
from scrape import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#scrapePage(anacUrl)

In [ ]:
situacaoVoos = []
fileCsvFinal = pd.DataFrame()
for ano in range(2015, 2020):
    print(f"Lendo os arquivos do ano {ano}")
    for i in range(1, 13):
            sep = [';', ',','\t']
            sepIndex = 0
            while True:
                try:
                    fileCsv = pd.read_csv(
                        f'data/{ano}-{i:02}.csv', encoding='ISO-8859-1', sep=sep[sepIndex], low_memory=False)
                    if ('internet' in fileCsv.columns[0]):
                        fileCsv.columns = fileCsv.iloc[0]
                        fileCsv = fileCsv[1:]
                    if len(fileCsv.columns) <= 1:
                        raise ValueError('sepIndex')
                    else:
                        try:
                            fileCsv = fileCsv.drop(columns='Data Prevista')
                        except:
                            pass
                        fileCsv.columns = ['ICAO Empresa Aérea', 'Número Voo', 'Código DI', 'Código Tipo Linha',
                                        'ICAO Aeródromo Origem', 'ICAO Aeródromo Destino', 'Partida Prevista',
                                        'Partida Real', 'Chegada Prevista', 'Chegada Real', 'Situação Voo',
                                        'Código Justificativa']
                    
                    fileCsv = fileCsv.assign(Ano = ano)
                    fileCsv = fileCsv.assign(Mes = i)
                    
                    fileCsvFinal = pd.concat([fileCsvFinal, fileCsv])
                    print(len(fileCsvFinal))
                    break
                except Exception as e:
                    if e.args[0] == 'sepIndex':
                        if (sepIndex < 3):
                            sepIndex += 1
                    else:
                        print(e)

Lendo os arquivos do ano 2015
114636
216629
327636
434556
542910
648608
760762
868806


In [ ]:
situacao = fileCsvFinal['Situação Voo'].str.upper()
situacaoVoos = situacao.value_counts()

In [ ]:
VoosPorAno = fileCsvFinal['Ano'].value_counts()
VoosPorAno

In [ ]:

fig, axs = plt.subplots(figsize=(15, 5))
VoosPorAno.plot(kind='bar')
plt.xticks(rotation=50)
labels = axs.get_xticklabels()
for label in labels:
    label.set_rotation(45)
plt.setp(axs,
yticks=[0, 200000, 400000, 600000, 800000, 1000000, 1200000])
plt.show()